In [8]:
%pip install --no-cache-dir torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 888.0/888.0 MB 2.2 MB/s eta 0:00:0000:0100:21
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 999.5 kB/s eta 0:00:0000:0100:07
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 1.6 MB/s eta 0:00:0000:0100:03
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 1.3 MB/s eta 0:00:0000:0100:04
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 2.9 MB/s eta 0:00:0000:0100:07
Note: you may need to restart the kernel to use updated packages.


In [9]:
%pip install -U sentence-transformers

  Using cached sentence_transformers-5.1.0-py3-none-any.whl (483 kB)
  Using cached transformers-4.55.2-py3-none-any.whl (11.3 MB)
  Using cached scikit_learn-1.7.1-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (9.7 MB)
  Using cached huggingface_hub-0.34.4-py3-none-any.whl (561 kB)
  Using cached tokenizers-0.21.4-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import pandas as pd
import pyterrier as pt
import numpy as np
from sentence_transformers import CrossEncoder

/home/killdollar/btp/Information-Retrieval/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
indexer = pt.IterDictIndexer('./indices/msmarco-passage')
index = indexer.path
bm25 = pt.terrier.Retriever(index, wmodel="BM25", num_results=100)

Java started (triggered by TerrierIndexer.__init__) and loaded: pyterrier.java, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]


In [3]:
ds_topics = pt.get_dataset('irds:msmarco-passage/trec-dl-2019/judged')
ds_text = pt.get_dataset('irds:msmarco-passage')
topics = ds_topics.get_topics()


In [4]:
def get_text_for_docnos(docnos, dataset):
    """Efficiently get text for specific document IDs"""
    doc_texts = {}
    
    # Convert all docnos to the format we'll try
    docno_variants = {}
    for docno in docnos:
        str_docno = str(docno)
        docno_variants[str_docno] = docno
        # Also try integer version if it's numeric
        if str_docno.isdigit():
            docno_variants[int(str_docno)] = docno
    
    # Only iterate through corpus until we find all needed documents
    needed_ids = set(docno_variants.keys())
    found_count = 0
    
    corpus_iter = dataset.get_corpus_iter()
    for doc in corpus_iter:
        doc_id = doc['docno']
        
        # Check if this document is one we need
        if doc_id in needed_ids:
            original_docno = docno_variants[doc_id]
            doc_texts[original_docno] = doc['text']
            found_count += 1
            
            # Stop early if we found all documents we need
            if found_count >= len(docnos):
                break
    
    # Fill in missing documents
    for docno in docnos:
        if docno not in doc_texts:
            doc_texts[docno] = "Document text not available"
    
    return doc_texts


In [5]:
import torch
try:
    print("Trying to load ColBERT model from HuggingFace...")
    from transformers import AutoModelForSequenceClassification, AutoTokenizer
    
    # Try to load ColBERT model
    model_name = "colbert-ir/colbertv2.0"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    
    def _colbert_apply(df):
        """Apply ColBERT scoring to query-document pairs"""
        pairs = list(zip(df['query'].values, df['text'].values))
        scores = []
        
        for query, doc in pairs:
            inputs = tokenizer(query, doc, return_tensors="pt", truncation=True, max_length=512)
            with torch.no_grad():
                outputs = model(**inputs)
                
            # Fix: Handle multi-element logits correctly
            # For binary classification models, use the second logit (positive class)
            if outputs.logits.size(1) > 1:
                # Use the second element (positive class score)
                score = outputs.logits[0, 1].item()
            else:
                # Use the single score if only one dimension
                score = outputs.logits.item()
                
            scores.append(score)
        
        return np.array(scores)
    
    # Create the PyTerrier transformer for reranking
    reranker = pt.apply.doc_score(_colbert_apply, batch_size=32)
    model_name_for_output = "colbert"
    print("Successfully loaded ColBERT model")
    
except Exception as e:
    print(f"Failed to load ColBERT: {e}")
    print("Falling back to Sentence Transformers MiniLM model...")
    # Rest of your fallback code remains the same

Trying to load ColBERT model from HuggingFace...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at colbert-ir/colbertv2.0 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Successfully loaded ColBERT model


In [6]:
def add_text_efficiently(df):
    """Add text to dataframe for only the documents in this batch"""
    df = df.copy()
    docnos = df['docno'].tolist()
    doc_texts = get_text_for_docnos(docnos, ds_text)
    df['text'] = df['docno'].map(doc_texts)
    return df

In [7]:
if reranker is not None:
    # BM25 -> Add Text -> Reranking
    retrieval_pipeline = bm25 >> pt.apply.generic(add_text_efficiently) >> reranker
else:
    # BM25 -> Add Text (no reranking)
    retrieval_pipeline = bm25 >> pt.apply.generic(add_text_efficiently)


In [8]:
all_results = []

for idx, row in topics.iterrows():
    query_id = row['qid']
    query_text = row['query']
    
    print(f"Processing query {idx+1}/{len(topics)}: {query_text}")
    
    # Get results from the pipeline
    results = retrieval_pipeline.search(query_text)
    
    # Take top 10 results
    top_results = results.head(10)
    
    # Add query information to results
    top_results['qid'] = query_id
    top_results['query'] = query_text
    
    # Add to collection
    all_results.append(top_results)
    
    # Show sample results for first query
    if idx == 0:
        print("\nSample results for first query:")
        print(top_results[['docno', 'score', 'text']].head(3))

# Step 8: Combine all results
combined_results = pd.concat(all_results) if all_results else pd.DataFrame()

# Step 9: Save results to CSV
output_file = f'{model_name_for_output}_results.csv'
combined_results.to_csv(output_file, index=False)

print(f"\nRetrieved {len(combined_results)} documents across {len(topics)} queries")
print(f"Results saved to {output_file}")

# Step 10: Show summary of results
print("\nTop documents for first few queries:")
print(combined_results.groupby('qid').head(1)[['qid', 'docno', 'score']].head(5))

Processing query 1/43: do goldfish grow


msmarco-passage documents: 100%|█████████▉| 8803136/8841823 [00:30<00:00, 285437.34it/s]
/tmp/ipykernel_225393/1915278885.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_results['qid'] = query_id
/tmp/ipykernel_225393/1915278885.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_results['query'] = query_text



Sample results for first query:
      docno     score                                               text
83  3024301  0.189813  Family Zoo: Growing up in Tasmania, her family...
98  3180514  0.079862  Koi Carp Worlds Largest Goldfish Bowl Goldfish...
71  2612491  0.070616  Goldfish growth rates can be massive for their...
Processing query 2/43: what is wifi vs bluetooth


msmarco-passage documents:  94%|█████████▍| 8301973/8841823 [00:26<00:01, 308702.46it/s]
/tmp/ipykernel_225393/1915278885.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_results['qid'] = query_id
/tmp/ipykernel_225393/1915278885.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_results['query'] = query_text


Processing query 3/43: why did the us volunterilay enter ww1


...

msmarco-passage documents:  99%|█████████▊| 8712732/8841823 [00:29<00:00, 298622.94it/s]
/tmp/ipykernel_225393/1915278885.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_results['qid'] = query_id
/tmp/ipykernel_225393/1915278885.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_results['query'] = query_text


Processing query 43/43: difference between a mcdouble and a double cheeseburger


msmarco-passage documents:  99%|█████████▉| 8795495/8841823 [00:29<00:00, 301062.21it/s]



Retrieved 425 documents across 43 queries
Results saved to colbert_results.csv

Top documents for first few queries:
        qid    docno     score
83   156493  3024301  0.189813
38  1110199   735472  0.121643
80  1063750  4047881  0.050373
41   130510  5685710  0.104719
52   489204  6461840  0.131342


/tmp/ipykernel_225393/1915278885.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_results['qid'] = query_id
/tmp/ipykernel_225393/1915278885.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_results['query'] = query_text
